In [2]:
import sys
import os
from sage.all import *
import random
from matplotlib.colors import CSS4_COLORS
from itertools import product
import sage.matrix.matrix_integer_dense_hnf as hnf
from sage.all import FractionField, vector, Matrix
from IPython.display import display, Math, Latex

import sage.typeset.character_art as character_art
from sage.repl.rich_output import get_display_manager
dm = get_display_manager()
dm.preferences.text = 'ascii_art'
import sage.typeset.character_art as character_art
character_art.MAX_WIDTH = 100

sage_folder = os.path.abspath("../sage")

if sage_folder not in sys.path:
    sys.path.append(sage_folder)

from loader import load_from_dir

# on import toutes les fonctions sages faites maison dont on a besoin
load_from_dir("../sage/nint.sage", globals())
load_from_dir("../sage/nmatrix.sage", globals())
load_from_dir("../sage/row_degree.sage", globals())
load_from_dir("../sage/shifted_row_degree.sage", globals())
load_from_dir("../sage/is_in_lattice.sage", globals())
load_from_dir("../sage/is_lattice_basis.sage", globals())
load_from_dir("../sage/random/random_full_rank_matrix.sage", globals())
load_from_dir("../sage/random/random_matrix_of_rank.sage", globals())
load_from_dir("../sage/random/random_GLZ_matrix.sage", globals())
load_from_dir("../sage/gram_schmidt.sage", globals())

### 1. Arrondi à l'entier le plus proche (*Nearest Integer*)

**Définition.**
Soit $x \in \mathbb{R}$. On définit l'entier le plus proche de $x$, noté $\lceil x\rfloor$, par la formule :

$$\lceil x\rfloor = \lfloor x + 1/2 \rfloor$$

> **Note sur la convention ($0.5$) :**
>
> Cette définition implique un arrondi vers l'infini positif pour les demi-entiers.
>
> *Exemple :* $0.5 \longrightarrow 1$ (et non $0$).

**Implémentation SageMath :**
Cette fonction est disponible sous le nom `nint`.

In [3]:
tests = [-4.23, 0.5, 1.499]

for t in tests:
    display(Math(rf"{t:.3f} \longrightarrow {nint(t)}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### 1.2. Matrice d'entiers la plus proche (*Nearest Matrix*)

**Définition.**
Soit $A \in \mathcal{M}_{n,m}(\mathbb{R})$ une matrice à coefficients réels.
On définit la matrice arrondie, notée $\lceil A \rfloor$, en appliquant la fonction *nearest integer* à chaque coefficient :

$$(\lceil A \rfloor)_{i,j} = \lceil A_{i,j} \rfloor$$

**Implémentation :**
La fonction `nmatrix(M)` applique `nint` élément par élément.

In [4]:
M_test = matrix([
    [-4.23, 0.5,  2.1],
    [ 3.99, -0.5, 10.0]
])

M_round = nmatrix(M_test)

display(Math(r"\text{Exemple de transformation :}"))

display(Math(
    rf"{latex(M_test)} \longrightarrow {latex(M_round)}"
))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### 2. Degré en ligne et Degré décalé

**Définition 1 (Degré en ligne).**
* Pour un vecteur ligne $\mathbf{m} = (m_1, \dots ,m_n) \in \mathbb{K}[x]^{1 \times n}$, on définit son **degré en ligne** par :
  $$\text{rdeg}(\mathbf{m}) = \max_{1 \leq j \leq n} \deg(m_j) \in \mathbb{Z}$$

* Pour une matrice $M$ composée de lignes $\mathbf{M_1}, \dots, \mathbf{M_r}$, on définit son **degré en ligne** par le vecteur :
  $$\text{rdeg}(M) = (\text{rdeg}(\mathbf{M_i}))_{1 \leq i \leq r} \in \mathbb{Z}^r$$

---

**Définition 2 (Degré décalé).**
Soit $\vec{\mathbf{s}} =(s_1, \dots, s_n) \in \mathbb{Z}^n$ un vecteur d'entiers appelé **vecteur de décalage**.

* Pour un vecteur ligne $\mathbf{m} = (m_1, \dots ,m_n)$, on définit son **degré en ligne $\vec{\mathbf{s}}$-décalé** par :
  $$\text{rdeg}_{\vec{\mathbf{s}}}(\mathbf{m}) = \max_{1 \leq j \leq n} (\deg(m_j) + s_j)$$

* Pour une matrice $M$, le **degré en ligne décalé** est le vecteur formé par les degrés décalés de ses lignes :
  $$\text{rdeg}_{\vec{\mathbf{s}}}(M) = \big( \text{rdeg}_{\vec{\mathbf{s}}}(\mathbf{M_i}) \big)_{1 \leq i \leq r} \in \mathbb{Z}^r$$

In [5]:
from IPython.display import display, Math

# 1. Préparation de l'anneau et des outils
R.<x> = ZZ[]
# Rappel : assurez-vous d'avoir chargé vos fonctions row_degree et shifted_row_degree avant !

# 2. Exemple de Matrice M (2 lignes, 2 colonnes)
# Ligne 1 : (x^2 + 1,  x)      -> degs: (2, 1)
# Ligne 2 : (5,        x^3-x)  -> degs: (0, 3)
M = matrix([
    [x^2 + 1,  x],
    [5,        x^3 - x]
])

# 3. Définition du vecteur de décalage s
# On ajoute 0 à la col 1, et 2 à la col 2
s = [0, 2]

# 4. Calculs
res_rdeg = row_degree(M)
res_shifted = shifted_row_degree(M, s)

# 5. Affichage "Pro" avec display Math
display(Math(r"\textbf{Exemple sur } \mathbb{Z}[x] :"))

# Affichage du degré simple
display(Math(
    rf"\text{{rdeg}}\left( {latex(M)} \right) = {latex(res_rdeg)}"
))

# Affichage du vecteur de décalage
display(Math(rf"\text{{Avec le vecteur de décalage }} \vec{{\mathbf{{s}}}} = {s} :"))

# Affichage du degré décalé
# Détail pour vérification visuelle :
# Ligne 1 décalée : max(2+0, 1+2) = max(2, 3) = 3
# Ligne 2 décalée : max(0+0, 3+2) = max(0, 5) = 5
display(Math(
    rf"\text{{rdeg}}_{{\vec{{\mathbf{{s}}}}}}(M) = {latex(res_shifted)}"
))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>